In [40]:
import arrow
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import pathlib
import xml.etree.ElementTree as ET
import xarray as xr
import codecs
import datetime

In [41]:
read_file_full_path: str = r'G:\05DATA\01TRAINING_DATA\FUB\MF01001\2024_local.csv'
df: pd.DataFrame = pd.read_csv(read_file_full_path)

In [42]:
df.head()

,time,longitude,latitude,WS,YBG
0,202401010000,120.595167,39.500333,10.4,1.9
1,202401010100,120.594667,39.500000,10.1,1.6
2,202401010200,120.594667,39.500333,7.9,1.5
3,202401010300,120.595167,39.500500,6.9,1.2
4,202401010400,120.595000,39.500500,5.7,1.2


In [43]:
df[-5:]

,time,longitude,latitude,WS,YBG
8882,202412311900,120.609667,39.503000,12.5,1.7
8883,202412312000,120.609667,39.503000,12.0,1.5
8884,202412312100,120.609667,39.503000,10.8,1.6
8885,202412312200,120.609667,39.503167,9.3,1.5
8886,202412312300,120.609500,39.503333,8.8,1.3


In [44]:
# 将time列的内容从int64 => str
df['time']=df['time'].astype(str)

In [45]:
df['time']=pd.to_datetime(df['time'], format='%Y%m%d%H%M')

In [47]:
# df['time']

In [22]:
mask=df['time']=='202401010000'

In [23]:
df.index[mask]

Index([0], dtype='int64')

In [19]:
type(df['time'][0])

numpy.int64

In [24]:
df.shape

(8887, 5)

#### step2:   
生成一整年的时间集合，时间步长1hour;并将原始dataframe的index以time series为索引生成标准索引，并填充nan

In [48]:
start_time='2024-01-01 00:00:00'
end_time='2024-12-31 23:00:00'

time_series=pd.date_range(start=start_time,end=end_time,freq='H')

C:\Users\evase\AppData\Local\Temp\ipykernel_47756\239949322.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series=pd.date_range(start=start_time,end=end_time,freq='H')


In [49]:
time_series

DatetimeIndex(['2024-01-01 00:00:00', '2024-01-01 01:00:00',
               '2024-01-01 02:00:00', '2024-01-01 03:00:00',
               '2024-01-01 04:00:00', '2024-01-01 05:00:00',
               '2024-01-01 06:00:00', '2024-01-01 07:00:00',
               '2024-01-01 08:00:00', '2024-01-01 09:00:00',
               ...
               '2024-12-31 14:00:00', '2024-12-31 15:00:00',
               '2024-12-31 16:00:00', '2024-12-31 17:00:00',
               '2024-12-31 18:00:00', '2024-12-31 19:00:00',
               '2024-12-31 20:00:00', '2024-12-31 21:00:00',
               '2024-12-31 22:00:00', '2024-12-31 23:00:00'],
              dtype='datetime64[ns]', length=8784, freq='h')

In [50]:
df.set_index('time',inplace=True)

In [51]:
df_reindexed = df.reindex(time_series)

In [58]:
df_reindexed.index.name = 'time'

In [59]:
df_reindexed.shape

(8784, 4)

In [34]:
df_reindexed[df_reindexed['YBG']==np.nan]

,longitude,latitude,WS,YBG


In [60]:
df.head()

,longitude,latitude,WS,YBG
time,,,,
2024-01-01 00:00:00,120.595167,39.500333,10.4,1.9
2024-01-01 01:00:00,120.594667,39.500000,10.1,1.6
2024-01-01 02:00:00,120.594667,39.500333,7.9,1.5
2024-01-01 03:00:00,120.595167,39.500500,6.9,1.2
2024-01-01 04:00:00,120.595000,39.500500,5.7,1.2


In [61]:
df_reindexed.head()

,longitude,latitude,WS,YBG
time,,,,
2024-01-01 00:00:00,120.595167,39.500333,10.4,1.9
2024-01-01 01:00:00,120.594667,39.500000,10.1,1.6
2024-01-01 02:00:00,120.594667,39.500333,7.9,1.5
2024-01-01 03:00:00,120.595167,39.500500,6.9,1.2
2024-01-01 04:00:00,120.595000,39.500500,5.7,1.2


In [63]:
df.index

DatetimeIndex(['2024-01-01 00:00:00', '2024-01-01 01:00:00',
               '2024-01-01 02:00:00', '2024-01-01 03:00:00',
               '2024-01-01 04:00:00', '2024-01-01 05:00:00',
               '2024-01-01 06:00:00', '2024-01-01 07:00:00',
               '2024-01-01 08:00:00', '2024-01-01 09:00:00',
               ...
               '2024-12-31 14:00:00', '2024-12-31 15:00:00',
               '2024-12-31 16:00:00', '2024-12-31 17:00:00',
               '2024-12-31 18:00:00', '2024-12-31 19:00:00',
               '2024-12-31 20:00:00', '2024-12-31 21:00:00',
               '2024-12-31 22:00:00', '2024-12-31 23:00:00'],
              dtype='datetime64[ns]', name='time', length=8887, freq=None)

In [64]:
df_reindexed.index

DatetimeIndex(['2024-01-01 00:00:00', '2024-01-01 01:00:00',
               '2024-01-01 02:00:00', '2024-01-01 03:00:00',
               '2024-01-01 04:00:00', '2024-01-01 05:00:00',
               '2024-01-01 06:00:00', '2024-01-01 07:00:00',
               '2024-01-01 08:00:00', '2024-01-01 09:00:00',
               ...
               '2024-12-31 14:00:00', '2024-12-31 15:00:00',
               '2024-12-31 16:00:00', '2024-12-31 17:00:00',
               '2024-12-31 18:00:00', '2024-12-31 19:00:00',
               '2024-12-31 20:00:00', '2024-12-31 21:00:00',
               '2024-12-31 22:00:00', '2024-12-31 23:00:00'],
              dtype='datetime64[ns]', name='time', length=8784, freq='h')

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

In [65]:
# df_reindexed.compare(df)

----

#### step3: 找到对应的start_time

In [67]:
target_time=pd.to_datetime('2024-01-01 00:00:00')
target_time

Timestamp('2024-01-01 00:00:00')

In [68]:
target_time=np.datetime64('2024-01-01 00:00:00')
target_time

numpy.datetime64('2024-01-01T00:00:00')

In [69]:
index=df_reindexed.index.get_loc(target_time)
index

0

In [70]:
df_reindexed[index:index+72]

,longitude,latitude,WS,YBG
time,,,,
2024-01-01 00:00:00,120.595167,39.500333,10.4,1.9
2024-01-01 01:00:00,120.594667,39.500000,10.1,1.6
2024-01-01 02:00:00,120.594667,39.500333,7.9,1.5
2024-01-01 03:00:00,120.595167,39.500500,6.9,1.2
2024-01-01 04:00:00,120.595000,39.500500,5.7,1.2
...,...,...,...,...
2024-01-03 19:00:00,120.595500,39.500500,8.7,0.4
2024-01-03 20:00:00,120.595500,39.500833,9.1,0.6
2024-01-03 21:00:00,120.595500,39.500500,9.7,0.9


In [74]:
issue_dt_list=pd.date_range(start=start_time, end=end_time, freq='12H')
issue_dt_list

C:\Users\evase\AppData\Local\Temp\ipykernel_47756\2992125679.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  issue_dt_list=pd.date_range(start=start_time, end=end_time, freq='12H')


DatetimeIndex(['2024-01-01 00:00:00', '2024-01-01 12:00:00',
               '2024-01-02 00:00:00', '2024-01-02 12:00:00',
               '2024-01-03 00:00:00', '2024-01-03 12:00:00',
               '2024-01-04 00:00:00', '2024-01-04 12:00:00',
               '2024-01-05 00:00:00', '2024-01-05 12:00:00',
               ...
               '2024-12-27 00:00:00', '2024-12-27 12:00:00',
               '2024-12-28 00:00:00', '2024-12-28 12:00:00',
               '2024-12-29 00:00:00', '2024-12-29 12:00:00',
               '2024-12-30 00:00:00', '2024-12-30 12:00:00',
               '2024-12-31 00:00:00', '2024-12-31 12:00:00'],
              dtype='datetime64[ns]', length=732, freq='12h')

In [73]:
issue_dt_list

DatetimeIndex(['2024-01-01 00:00:00', '2024-01-01 12:00:00',
               '2024-01-02 00:00:00', '2024-01-02 12:00:00',
               '2024-01-03 00:00:00', '2024-01-03 12:00:00',
               '2024-01-04 00:00:00', '2024-01-04 12:00:00',
               '2024-01-05 00:00:00', '2024-01-05 12:00:00',
               ...
               '2024-12-27 00:00:00', '2024-12-27 12:00:00',
               '2024-12-28 00:00:00', '2024-12-28 12:00:00',
               '2024-12-29 00:00:00', '2024-12-29 12:00:00',
               '2024-12-30 00:00:00', '2024-12-30 12:00:00',
               '2024-12-31 00:00:00', '2024-12-31 12:00:00'],
              dtype='datetime64[ns]', length=732, freq='12h')